In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [2]:
get_weather('berlin')

20.0

In [3]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Fetch the Weather of the City",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Which city's weather ?"
            }
        },
        "required": ['city'],
        "additionalProperties": False
    }
}

In [5]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-26 23:30:07--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3495 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py.2’

chat_assistant.py.2 100%[===================>]   3.41K  --.-KB/s    in 0s      

2025-07-26 23:30:07 (41.8 MB/s) - ‘chat_assistant.py.2’ saved [3495/3495]

